<h2>ATL Braves Data Wrangling</h2>

In [1]:
import requests
import pandas as pd
import sqlite3
import openpyxl

make sure the connection to the API is valid

In [3]:
url = 'https://statsapi.mlb.com/api/v1/stats?stats=season&group=pitching&playerPool=all&season=2018&teamId=144'
response = requests.get(url)
print(response.status_code)

data = response.json()

200


connect to the SQLite database

In [5]:
try:

    connection = sqlite3.connect('C:/Users/taylo/OneDrive/Documents/Personal/Projects/ATL Braves/main')
    print('Conncetion successful')
except sqlite3.Error as err:
    print("Failed to connect to db: ", err)


cursor = connection.cursor()

Conncetion successful


load the data from the API into a pandas dataframe named statsAPIdf, then load the data from the database into a pandas dataframe named pitch_by_pitch_df . <br>
Once loaded, the dataframe can be exported to a CSV file. 

In [7]:
pitch_by_pitch_df = pd.read_sql('SELECT * FROM PITCHBYPITCH', connection)
statsAPIdf = pd.json_normalize(data, record_path=['stats', 'splits'])
statsAPIdf = statsAPIdf.rename(columns=lambda x: x.replace('stat.', ''))
statsAPIdf.to_excel('C:/Users/taylo/OneDrive/Documents/Personal/Projects/ATL Braves/statsAPImlb.xlsx')

In [25]:
pitch_by_pitch_df.head()

,GameKey,GameDate,PitcherName,PitcherID,PA_OF_INNING,PITCH_OF_PA,INNING,TOP_BOT,BALLS,STRIKES,SWING_TAKE,IS_SINGLE,IS_DOUBLE,IS_TRIPLE,IS_HOMERUN,IS_HIT,IS_OUT,LAST_PITCH_OF_PA,IS_STRIKEOUT
0,298505,6/6/2018,"Venters, Jonny",458924,5,3,1,2,2,0,take,0,0,0,0,0,0,0,0
1,298505,6/6/2018,"Venters, Jonny",458924,5,4,1,2,2,1,take,0,0,0,0,0,0,0,0
2,298505,6/6/2018,"Venters, Jonny",458924,2,3,1,2,2,0,take,0,0,0,0,0,0,0,0
3,298505,6/6/2018,"Venters, Jonny",458924,6,1,1,2,0,0,take,0,0,0,0,0,0,0,0
4,298505,6/6/2018,"Venters, Jonny",458924,1,1,1,2,0,0,swing,1,0,0,0,1,0,1,0


In [27]:
statsAPIdf.head()

,season,rank,gamesPlayed,gamesStarted,groundOuts,airOuts,runs,doubles,triples,homeRuns,...,league.id,league.name,league.link,sport.id,sport.link,sport.abbreviation,position.code,position.name,position.type,position.abbreviation
0,2018,1,27,0,26,24,8,2,1,1,...,104,NL,/api/v1/league/104,1,/api/v1/sports/1,MLB,1,Pitcher,Pitcher,P
1,2018,2,14,0,9,14,3,1,0,2,...,104,NL,/api/v1/league/104,1,/api/v1/sports/1,MLB,1,Pitcher,Pitcher,P
2,2018,3,39,0,28,43,9,8,0,4,...,104,NL,/api/v1/league/104,1,/api/v1/sports/1,MLB,1,Pitcher,Pitcher,P
3,2018,4,25,24,142,124,48,21,2,15,...,104,NL,/api/v1/league/104,1,/api/v1/sports/1,MLB,1,Pitcher,Pitcher,P
4,2018,5,31,31,161,176,65,25,5,17,...,104,NL,/api/v1/league/104,1,/api/v1/sports/1,MLB,1,Pitcher,Pitcher,P


In [94]:
len(pitch_by_pitch_df)

2177

In [96]:
len(statsAPIdf)

35

**Note** The API data conatins 35 rows of records while the pbp table contains over 2k

In [48]:
api_col = set(statsAPIdf.columns)
pbp_col = set(pitch_by_pitch_df.columns)

The following below are the columns that are in the API that are not in the pitchbypitch table

In [51]:
api_col.difference(pbp_col)

{'airOuts',
 'atBats',
 'avg',
 'balks',
 'baseOnBalls',
 'battersFaced',
 'blownSaves',
 'catchersInterference',
 'caughtStealing',
 'completeGames',
 'doubles',
 'earnedRuns',
 'era',
 'gamesFinished',
 'gamesPitched',
 'gamesPlayed',
 'gamesStarted',
 'groundIntoDoublePlay',
 'groundOuts',
 'groundOutsToAirouts',
 'hitBatsmen',
 'hitByPitch',
 'hits',
 'hitsPer9Inn',
 'holds',
 'homeRuns',
 'homeRunsPer9',
 'inheritedRunners',
 'inheritedRunnersScored',
 'inningsPitched',
 'intentionalWalks',
 'league.id',
 'league.link',
 'league.name',
 'losses',
 'numberOfPitches',
 'obp',
 'ops',
 'outs',
 'pickoffs',
 'pitchesPerInning',
 'player.firstName',
 'player.fullName',
 'player.id',
 'player.lastName',
 'player.link',
 'position.abbreviation',
 'position.code',
 'position.name',
 'position.type',
 'rank',
 'runs',
 'runsScoredPer9',
 'sacBunts',
 'sacFlies',
 'saveOpportunities',
 'saves',
 'season',
 'shutouts',
 'slg',
 'sport.abbreviation',
 'sport.id',
 'sport.link',
 'stolenBasePe

The following below are the columns that are in the pbp table that are not in the API

In [21]:
pbp_col.difference(api_col)

{'BALLS',
 'GameDate',
 'GameKey',
 'INNING',
 'IS_DOUBLE',
 'IS_HIT',
 'IS_HOMERUN',
 'IS_OUT',
 'IS_SINGLE',
 'IS_STRIKEOUT',
 'IS_TRIPLE',
 'LAST_PITCH_OF_PA',
 'PA_OF_INNING',
 'PITCH_OF_PA',
 'PitcherID',
 'PitcherName',
 'STRIKES',
 'SWING_TAKE',
 'TOP_BOT'}

**Note** The difference in columns suggests that the pbp table is on a pitch level and the API is tracking stats on a more high level perhaps a season level

In [42]:
pitch_by_pitch_df.dtypes

GameKey             object
GameDate            object
PitcherName         object
PitcherID           object
PA_OF_INNING        object
PITCH_OF_PA         object
INNING              object
TOP_BOT             object
BALLS               object
STRIKES             object
SWING_TAKE          object
IS_SINGLE           object
IS_DOUBLE           object
IS_TRIPLE           object
IS_HOMERUN          object
IS_HIT              object
IS_OUT              object
LAST_PITCH_OF_PA    object
IS_STRIKEOUT        object
dtype: object

In [77]:
statsAPIdf.dtypes

season                   object
rank                      int64
gamesPlayed               int64
gamesStarted              int64
groundOuts                int64
                          ...  
sport.abbreviation       object
position.code            object
position.name            object
position.type            object
position.abbreviation    object
Length: 80, dtype: object

**Note** The data from the API contains various data types while the pbp table only contains object data type.

In [82]:
statsAPIdf.isnull().sum()

season                   0
rank                     0
gamesPlayed              0
gamesStarted             0
groundOuts               0
                        ..
sport.abbreviation       0
position.code            0
position.name            0
position.type            0
position.abbreviation    0
Length: 80, dtype: int64

In [46]:
pitch_by_pitch_df.isnull().sum()

GameKey             0
GameDate            0
PitcherName         0
PitcherID           0
PA_OF_INNING        0
PITCH_OF_PA         0
INNING              0
TOP_BOT             0
BALLS               0
STRIKES             0
SWING_TAKE          0
IS_SINGLE           0
IS_DOUBLE           0
IS_TRIPLE           0
IS_HOMERUN          0
IS_HIT              0
IS_OUT              0
LAST_PITCH_OF_PA    0
IS_STRIKEOUT        0
dtype: int64

<h4>Next Steps to Develop Audit System</h4>

The first step in developing an audit system for me would be to create documentation. We need to take note of how the audit system was developed and what information it contains. Data dictionaries on each table would be helpful for better understanding and reduce ramp up time.

Next we we need to validate the data. Making sure all columns are formatted with the correct data type (making sure stats are numeric to ensure proper mathematical operations). Column names should make the schema of whatever table the records are to be inserted.  

Additionally, we can add a visualization component to better communicate findings and identify trends and/or errors in the data.

Lastly, We want to system to be able to pull data real time without having to manually do so. Therefore incorporating an automated process to check for new game data to add to the database will be helpful. 